# Database Connection Test

Quick test to verify SQL Server connectivity without loading all helpers.

In [ ]:
import pyodbc
import os
import pandas as pd

In [ ]:
# Connection parameters
server = os.getenv('DB_SERVER', 'sqlserver')
database = 'master'  # Start with master to test basic connectivity
username = os.getenv('DB_USER', 'sa')
password = os.getenv('DB_PASSWORD', 'IRP_Pass2024!')
port = os.getenv('DB_PORT', '1433')

print("Connection Parameters:")
print(f"  Server: {server}")
print(f"  Port: {port}")
print(f"  User: {username}")
print(f"  Database: {database}")

In [ ]:
# Test basic connectivity
print("\nTesting connection to SQL Server...\n")

try:
    conn_str = (
        f"DRIVER={{ODBC Driver 18 for SQL Server}};"
        f"SERVER={server},{port};"
        f"DATABASE={database};"
        f"UID={username};"
        f"PWD={password};"
        f"TrustServerCertificate=yes;"
    )
    
    conn = pyodbc.connect(conn_str, timeout=5)
    cursor = conn.cursor()
    
    # Test query
    cursor.execute("SELECT @@VERSION")
    version = cursor.fetchone()[0]
    
    print("✅ Connection successful!")
    print(f"\nServer version: {version[:100]}...")
    
    conn.close()
    
except Exception as e:
    print(f"❌ Connection failed: {e}")
    print("\nTroubleshooting steps:")
    print("1. Check if SQL Server container is running: docker ps")
    print("2. Wait 30 seconds for SQL Server to fully start")
    print("3. Check Docker logs: docker logs irp-sqlserver")

In [ ]:
# Test IRP_DB database
print("\nTesting IRP_DB database...\n")

try:
    conn_str = (
        f"DRIVER={{ODBC Driver 18 for SQL Server}};"
        f"SERVER={server},{port};"
        f"DATABASE=IRP_DB;"
        f"UID={username};"
        f"PWD={password};"
        f"TrustServerCertificate=yes;"
    )
    
    conn = pyodbc.connect(conn_str, timeout=5)
    
    # Check if tables exist
    query = """
    SELECT TABLE_NAME 
    FROM INFORMATION_SCHEMA.TABLES 
    WHERE TABLE_TYPE = 'BASE TABLE' 
    AND TABLE_NAME LIKE 'irp_%'
    ORDER BY TABLE_NAME
    """
    
    df = pd.read_sql_query(query, conn)
    
    if not df.empty:
        print("✅ IRP_DB database found with tables:")
        for table in df['TABLE_NAME']:
            print(f"  - {table}")
    else:
        print("⚠️  IRP_DB exists but no tables found")
        print("   Run Database_Admin.ipynb to initialize tables")
    
    conn.close()
    
except pyodbc.ProgrammingError as e:
    if 'Cannot open database' in str(e):
        print("⚠️  IRP_DB database not found")
        print("   Run Database_Admin.ipynb to create and initialize")
    else:
        print(f"❌ Error: {e}")
except Exception as e:
    print(f"❌ Connection failed: {e}")

In [ ]:
# List all databases
print("\nAvailable databases:")

try:
    conn_str = (
        f"DRIVER={{ODBC Driver 18 for SQL Server}};"
        f"SERVER={server},{port};"
        f"DATABASE=master;"
        f"UID={username};"
        f"PWD={password};"
        f"TrustServerCertificate=yes;"
    )
    
    conn = pyodbc.connect(conn_str)
    df = pd.read_sql_query("SELECT name FROM sys.databases ORDER BY name", conn)
    
    for db in df['name']:
        if db not in ['master', 'tempdb', 'model', 'msdb']:
            print(f"  📁 {db}")
    
    conn.close()
    
except Exception as e:
    print(f"❌ Error listing databases: {e}")